<a href="https://colab.research.google.com/github/Lemon-Aki/studyAI/blob/main/chapter05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
#테스트 세트로 평가를 하다보면 테스트 세트에 잘 맞는 모델이 만들어지는 문제
#검증 세트 : 훈련 세트를 나누어 검증용으로 사용하는 데이터
#검증 세트 만들기
#판다스로 csv파일로부터 데이터를 읽어들여 데이터프레임 생성
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

wine.head()

#입력 데이터 생성
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [57]:
#입력 데이터에서 훈련 세트, 테스트 세트 생성
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target,test_size=0.2, random_state=42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [58]:
#훈련 세트를 나눠서 훈련용 세트와 검증 세트 생성
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [59]:
#훈련용 세트로 결정 트리 모델 생성
#과대적합문제 발생, 매개변수를 바꿔서 더 좋은 모델을 찾고 싶음
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [60]:
#교차 검증 : 검증 세트를 떼어 평가하는 과정을 여러번 반복, 전체 훈련 세트를 n개(보통 5, 10)의 폴드로 나눈 후, 번갈아가면서 검증
#그 후 각 검증의 점수를 평균을 내어 최종 검증 점수를 획득
#폴드 : n등분 된 데이터의 집합
#cross_validate() : 교차 검증 클래스, 기본 5-폴드 교차 검증 수행, cv 파라미터로 폴드 수를 변경 가능, 훈련 세트를 섞지 않음
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0083921 , 0.00746179, 0.007622  , 0.00766611, 0.00722504]), 'score_time': array([0.00128675, 0.00072312, 0.00072837, 0.00076866, 0.00074911]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [61]:
#교차 검증의 최종 점수를 획득(평균점수)
import numpy as np
mean = np.mean(scores['test_score'])
print(mean)

0.855300214703487


In [62]:
#cross_validate()는 훈련 세트를 섞지않고 폴드를 만들기 때문에 그 전에 train_test_split()으로 섞던가 해야함
#분할기(splitter) : 훈련 세트를 섞는 변수, 회귀 모델일 경우 KFold분할기, 분류 모델일 경우 StratifiedKFold 클래스 사용
#StratifiedKFold 클래스 : 분류 모델에서 훈련 세트를 섞기 위한 함수
#위의 
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [63]:
#n_splits : 몇개의 폴드로 나눠 교차 검증을 할지 정하는 파라미터
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [64]:
#결정 트리의 파라미터 값을 바꿔가며 가장 좋은 모델 찾기
#하이퍼파라미터 : 모델이 학습할 수 없는 사용자가 지정해야만 하는 파라미터
#그리드서치(GridSearchCV) : 하이퍼파라미터 탐색과 교차 검증을 한번에 가능한 클래스, 
#  가장 좋은 검증 점수의 파라미터 조합을 구한후, 그 조합으로 모델을 최종적으로 훈련해 줌
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

#cv(폴드갯수)는 기본 5개 x 파라미터가 5개배열이기 때문에 25번의 교차 검증 실시
#n_jobs : 병렬 실행에 사용할 CPU 코어 수 지정, -1은 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [65]:
#best_estimator_ : 25개의 모델중 검증 점수가 가장 높은 모델을 저장하고 있는 속성
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

#best_params_ : 가장 높은 모델의 파라미터를 저장하고 있는 속성
print(gs.best_params_)

#cv_results_['mean_test_score'] : 교차 검증의 평균 점수를 저장하고 있는 키
print(gs.cv_results_['mean_test_score'])

#argmax() : 가장 큰 값의 인덱스를 추출
#params 키의 해당 인덱스에 저장된 파라미터 출력(best_params_와 동일)
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [66]:
#9 * 15 * 10 * 5폴드 = 6750모델 생성
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5, 20, 1),
          'min_samples_split':range(2, 100, 10)
          }
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

#최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [67]:
#최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [68]:
#랜덤 서치 : 매개변수의 값의 리스트를 전달하는 것이 아닌, 매개변수를 샘플링 가능한 확률 분포 객체 전달
#확률 분포 : uniform이나 randint등 균등 분포에서 샘플링(주어진 범위에서 고르게 값을 취득)
#randint : 정수값, uniform : 실수값
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

#1000번 샘플링에서 각 숫자의 개수 확인
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 87,  85, 100, 122, 114,  96,  97, 108,  94,  97]))

In [69]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.51812179, 0.73836321, 0.85303283, 0.79340439, 0.83512772,
       0.66551159, 0.27130831, 0.90618269, 0.48549426, 0.02389726])

In [70]:
#확률 분포 객체로 탐색할 파라미터 생성
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

In [71]:
#RandomizedSearchCV : sklearn의 랜덤 서치 클래스
#n_iter : 샘플링 횟수
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [72]:
#최적의 파라미터 조합 확인
print(gs.best_params_)

#최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [73]:
#최적의 파라미터를 적용한 최적 모델로 결정 트리 모델 생성
#여기서 처음으로 테스트 세트 사용해서 평가
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
